In [1]:
from timeit import timeit

import numpy as np

from muse import r_est_naive, r_est_jackknife

AIR_TEMP = 20 # in degrees C

# --- AUDIO SAMPLING PARAMS ---
N = 100000 # number of samples
SAMPLE_RATE = 1250
dt = 1 / SAMPLE_RATE

# frequency range of interest, used to filter out frequencies outside the range
f_lo = 0
f_hi = 62500

N_MICS = 4

# --- SPATIAL PARAMS ---
ROOM_DIM = (3.3, 2.17, 0.53)
X_DIM, Y_DIM, Z_DIM = ROOM_DIM
offset = 0.1

# (N_MICS, 3) array storing position of each mic
MIC_POSITIONS = np.array([
    [X_DIM-offset, Y_DIM-offset, Z_DIM-offset],
    [offset, offset, Z_DIM-offset],
    [offset, Y_DIM-offset, Z_DIM-offset],
    [X_DIM-offset, offset, Z_DIM-offset]
])

# param describing our desired resolution when calculating RSRP values
# across the floor of the arena
GRID_RESOLUTION = 0.025

rng = np.random.default_rng(seed=2022)
# take samples more on the order of actual data (10e5)
audio_input = rng.random(size=(N_MICS, N))

# def r_est_jackknife_slow(	
#     v, fs, f_lo, f_hi, temp, x_len, y_len, resolution, mic_positions,
#     ):
#     """Naive way to implement the Jackknife procedure."""
#     r_estimates = []	
#     rsrp_grids = []	

#     N_MICS = v.shape[0]	

#     for i in range(N_MICS):	
#         # remove mic i from audio and mic position arrays	
#         v_omitted = np.delete(v, i, axis=0)	
#         mic_pos_omitted = np.delete(mic_positions, i, axis=0)	
#         # calculate estimates	
#         r_est, rsrp_grid = r_est_naive(	
#             v_omitted, fs, f_lo, f_hi, temp,	
#             x_len, y_len, resolution, mic_pos_omitted	
#         )	
#         r_estimates.append(r_est)	
#         rsrp_grids.append(rsrp_grid)	

#     avg_est = np.mean(r_estimates, axis=0)	

#     return avg_est, r_estimates, rsrp_grids

# naive_time = timeit(
#     lambda: r_est_jackknife_slow(
#         audio_input, SAMPLE_RATE, f_lo, f_hi, AIR_TEMP,
#         X_DIM, Y_DIM, GRID_RESOLUTION, MIC_POSITIONS
#         ),
#     number=50) / 50

# new_time = timeit(
#     lambda: r_est_jackknife(
#         audio_input, SAMPLE_RATE, f_lo, f_hi, AIR_TEMP,
#         X_DIM, Y_DIM, GRID_RESOLUTION, MIC_POSITIONS
#         ),
#     number=50) / 50

# print(f'naive time: {naive_time}  |  new time: {new_time}')

In [2]:
import cProfile
import pickle

PATH_TO_DATA = 'dataset.pickle'
def wrapper():
    with open(PATH_TO_DATA, 'rb') as f:
        dataset = pickle.load(f)
        for i, d in enumerate(dataset):
            r_est_jackknife(
                d.signal, SAMPLE_RATE, f_lo, f_hi, AIR_TEMP,
                X_DIM, Y_DIM, GRID_RESOLUTION, MIC_POSITIONS
                )

cProfile.run('wrapper()')
# to_run = """r_est_jackknife(
#     audio_input, SAMPLE_RATE, f_lo, f_hi, AIR_TEMP,
#     X_DIM, Y_DIM, GRID_RESOLUTION, MIC_POSITIONS
#     )"""

# cProfile.runctx(to_run, globals(), locals())

         872101 function calls (852775 primitive calls) in 22.485 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.112    0.112   22.486   22.486 880999643.py:5(wrapper)
      160    0.001    0.000    0.019    0.000 <__array_function__ internals>:177(amax)
      160    0.000    0.000    0.011    0.000 <__array_function__ internals>:177(amin)
      961    0.002    0.000    0.041    0.000 <__array_function__ internals>:177(any)
      800    0.002    0.000    0.025    0.000 <__array_function__ internals>:177(argmax)
        4    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(atleast_1d)
      160    0.001    0.000    0.017    0.000 <__array_function__ internals>:177(broadcast_arrays)
      320    0.001    0.000    0.024    0.000 <__array_function__ internals>:177(concatenate)
        1    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(copyto)
     1120    0.003    0.000  